<a href="https://colab.research.google.com/github/MarioMarkov/cars-predict/blob/master/cars_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

#Load data
data = pd.read_csv('/content/cars-data.csv')

# Remove Id column
data = data[1:]

#data.price = np.log10(data.price) 

# Train set without price col
X = data.drop(['price'],axis='columns')

# Train set price col
y = data.price

# Encoding string columns to numeric
nominal_cols = X.select_dtypes(include='object')
le = preprocessing.LabelEncoder()
for col in nominal_cols:
  X[col] = le.fit_transform(X[col])

# Space of possible hyperparameters
space = dict()
space['n_estimators'] = range(100,600,100)
space['learning_rate'] = [0.01,0.02,0.03,0.04,0.05,0.1,0.2,0.3,0.4,0.5]

# Defining model to try hyperparameters from space on
model = XGBRegressor(objective='reg:squarederror')

# Defining cross-validation
cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1)

# Define final model
search = GridSearchCV(model, space, scoring='neg_mean_absolute_error',cv=cv,verbose=1, n_jobs=2, return_train_score=True)

#X_train, X_valid, y_train,y_valid = train_test_split(X,y,random_state=1,test_size=0.2)

# Fit model
search.fit(X,y)

#Calculate error 
mae = -1 * cross_val_score(search, X, y,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
#preds = pd.DataFrame(model.predict(X_valid))
#mae = mean_absolute_error(preds,y_valid)
print(mae.mean())
#preds = 10 ** preds
#preds.to_csv("preds.csv",index = False)


#y_valid = 10 ** y_valid
#y_valid.to_csv("real.csv",index = False)



Fitting 9 folds for each of 50 candidates, totalling 450 fits
Fitting 9 folds for each of 50 candidates, totalling 450 fits
Fitting 9 folds for each of 50 candidates, totalling 450 fits
Fitting 9 folds for each of 50 candidates, totalling 450 fits


In [12]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import LocalOutlierFactor



data = pd.read_csv('/content/cars-data.csv')
data = data[1:]

X = data.drop(['price'],axis='columns')
y = data.price

lof = LocalOutlierFactor()
nominal_cols = X.select_dtypes(include='object')
yhat = lof.fit_predict(X.select_dtypes(include='object'))

ValueError: ignored